In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import mlflow.sklearn
import joblib
import os




In [2]:
# Load preprocessed data
df = pd.read_csv('data/processed_data.csv')


In [3]:
# Use already one-hot encoded columns
X = df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S']]
y = df['Survived']

In [4]:
# Handle missing values (if any remain)
X['Age'] = X['Age'].fillna(X['Age'].mean())
X['Fare'] = X['Fare'].fillna(X['Fare'].mean())

/tmp/ipykernel_12024/57309919.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'] = X['Age'].fillna(X['Age'].mean())
/tmp/ipykernel_12024/57309919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Fare'] = X['Fare'].fillna(X['Fare'].mean())


In [5]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Training and logging with MLflow
mlflow.set_experiment("Titanic_LogReg")


with mlflow.start_run():
   model = LogisticRegression(max_iter=500)
   model.fit(X_train, y_train)


   y_pred = model.predict(X_test)
   accuracy = accuracy_score(y_test, y_pred)


   mlflow.log_param("model_type", "LogisticRegression")
   mlflow.log_param("max_iter", 500)
   mlflow.log_metric("accuracy", accuracy)
   mlflow.sklearn.log_model(model, "model")


   print("Accuracy:", accuracy)
   print(classification_report(y_test, y_pred))


   os.makedirs('model', exist_ok=True)
   joblib.dump(model, 'model/logreg_model.pkl')




2025/06/06 16:26:24 INFO mlflow.tracking.fluent: Experiment with name 'Titanic_LogReg' does not exist. Creating a new experiment.
2025/06/06 16:26:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

